# A toy example
# Convert to Julia file by jupytext --to jl ToyExample.ipynb

In [1]:
include("../src/DSI.jl")
include("../src/Zono_utils.jl")
include("../src/PZono.jl")
include("../src/DSZ.jl")
include("../src/marcelstuff.jl")

pbox_from_data (generic function with 1 method)

In [2]:
print_figures = true;

In [ ]:
# Toy example: defining the network
W1 = [1 -1.0; 1.0 1.]
b = [0.0; 0.0]
W2 = [1 -1.0; 1.0 1.]
L1 = Layer(W1, b, ReLU())
L2 = Layer(W2, b, Id())
full_net = Network([L1; L2])

# input range
x = [interval(-2,2),interval(-1.,1.)]

2-element Vector{Interval{Float64}}:
 [-2, 2]
 [-1, 1]

In [8]:
using PyPlot
ProbabilityBoundsAnalysis.plot(x[1])
PyPlot.savefig("input_range.png")

In [330]:
u = Base.rand()
a = uniform(0, 1)
s = cut(a, u)
typeof(s)

Interval{Float64}

In [335]:
y = sample_from_pboxes(x, 1000)
y_prop = []
for i in 1:length(y)
    aux = compute_output(full_net, y[i])
    push!(y_prop, aux)
end
y_prop1 = [y_prop[i][1] for i in 1:length(y_prop)]
println(maximum(y_prop1))
y_prop2 = [y_prop[i][2] for i in 1:length(y_prop)]
println(maximum(y_prop2))

NaN
NaN


In [337]:
#pyplot()
# Running the example with 2 focal elements
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
px = uniform.(inf.(x),sup.(x)) 
# Printing the pbox
pz = pbox_approximate_nnet(full_net,px,true) # independent inputs
if print_figures
    #ProbabilityBoundsAnalysis.plot(px[1])
    #ProbabilityBoundsAnalysis.plot(px[2])
    ProbabilityBoundsAnalysis.plot(pz[1])
    sorted_data, cdf = empirical_cdf(y_prop1)
    plot(sorted_data, cdf, marker="o", linestyle="-", label="Empirical CDF")
    PyPlot.savefig("plot1.png")

    ProbabilityBoundsAnalysis.plot(pz[2])
    sorted_data, cdf, low, up = pbox_from_data(y_prop2)
    plot(sorted_data, cdf, marker="o", linestyle="-", label="Empirical CDF")
    plot(sorted_data, low, marker="o", linestyle="-", label="Empirical CDF")
    plot(sorted_data, up, marker="o", linestyle="-", label="Empirical CDF")
    PyPlot.savefig("plot2.png")
end

In [190]:
function Compare_analyzes(px::Vector{pbox})


    println("DSI analysis")
    @time dsi_pz = pbox_approximate_nnet(full_net,px,true)  # true for independent inputs
    print("dsi_pz[1] <= -2 : ",dsi_pz[1] <= -2,"\n")
    print("dsi_pz[2] >= 2 : ",dsi_pz[2] >= 2,"\n")
    

    
    println("Probabilistic zonotope analysis")
    @time pzono_pz = PZono_approximate_nnet(full_net,px,true) # true for independent inputs
    print("pzono_pz[1] <= -2 : ",pzono_pz[1] <= -2,"\n")
    print("pzono_pz[2] >= 2 : ",pzono_pz[2] >= 2,"\n")
    ProbabilityBoundsAnalysis.plot(pzono_pz[1])
    PyPlot.savefig("huh.png")

    println("DSZ analysis")
    mat_spec_1 = [[1.0 0.0]
                  [0.0 -1.0]]
    rhs_spec_1 = [-2.0, -2.0]
    @time vec_proba = dsz_approximate_nnet_and_condition_nostorage(full_net, px, mat_spec_1,rhs_spec_1,true) 
    println("dsz_pz[1] <= -2 :", vec_proba[1])
    println("dsz_pz[2] >= 2 :", vec_proba[2])
    #println("dsz_pz[1] <= -2 and dsz_pz[2] >= 2 :", vec_proba[3])

end

Compare_analyzes (generic function with 1 method)

In [ ]:
#ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
#px = [(normal(interval(0,1),1)-0.5)/3.5902*2.,(normal(interval(0,1),1)/2.0-0.25)/1.7951]

println("Inputs from Uniform law (independent components)")
println("-----------------------")
for nb_discretization_steps in (2,10,100, 1000)
    println("Number of focal elements on the inputs = ",nb_discretization_steps)
    ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
    px = uniform.(inf.(x),sup.(x)) 
    Compare_analyzes(px)
    println()
end

println("************************************************************************************************************************")
println()
println("Inputs from truncated Normal law (independent components)")
println("-------------------------------")
for nb_discretization_steps in (10,100, 1000)
    println("Number of focal elements on the inputs = ",nb_discretization_steps)
    px = init_pbox_Normal(inf.(x),sup.(x),nb_discretization_steps,true) # truncated normal law
    Compare_analyzes(px)
    println()
end